# Project setup

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from importlib import reload
import src.modules.data_cleaning as dc

# Reload the data_cleaning module to ensure that any changes made to it are reflected in this notebook
reload(dc)

<module 'src.modules.data_cleaning' from '/Users/ds/data_science/demand_prediction_data/src/modules/data_cleaning.py'>

In [3]:
import os

global DATA_FOLDER, SRC_FOLDER, MODULES_FOLDER, TESTS_FOLDER, OUTPUT_FOLDER, FIGURES_FOLDER

# Define folder paths as global variables
DATA_FOLDER = "data/"
SRC_FOLDER = "src/"
MODULES_FOLDER = "src/modules/"
TESTS_FOLDER = "src/tests/"
OUTPUT_FOLDER = "output/"
FIGURES_FOLDER = "output/figures/"

# folders = [
#     DATA_FOLDER,
#     SRC_FOLDER,
#     MODULES_FOLDER,
#     TESTS_FOLDER,
#     OUTPUT_FOLDER,
#     FIGURES_FOLDER,
# ]

# for folder in folders:
#     if not os.path.exists(folder):
#         os.makedirs(folder)
#         print(f"Created folder: {folder}")
#     else:
#         print(f"Folder already exists: {folder}")


In [4]:
global DATA_PATH, DATA_ORIGINAL_PATH, DATA_GENERATED_PATH, IMAGE_GENERATED_PATH

DATA_PATH = DATA_FOLDER
DATA_ORIGINAL_PATH = DATA_FOLDER
DATA_GENERATED_PATH = OUTPUT_FOLDER
IMAGE_GENERATED_PATH = FIGURES_FOLDER


global COLORS

COLORS = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']

In [5]:
global LSTM1_OUT_PATH
LSTM1_OUT_PATH = DATA_GENERATED_PATH + 'LSTM_1/'

# EDA

## Accessing the Database

In [6]:
# read the parquet file
import pandas as pd
df = pd.read_parquet(DATA_GENERATED_PATH + 'ETL_finished.parquet')

In [7]:
# min max in the df.index
df.index.min()
df.index.max()

Timestamp('2019-06-01 00:00:00')

Timestamp('2022-01-14 00:00:00')

In [8]:


# # Check pairing for all combinations
# dc.check_all_column_pairings(df)



Output:

* brand_id and brand_name are paired 1-to-1
* group0_id and group0 are paired 1-to-1
* group1_id and group1 are paired 1-to-1
* group2_id and group2 are paired 1-to-1

In [9]:

dc.print_df_with_dtypes(df, max_rows=200, print_on=True, data_generated_path=DATA_GENERATED_PATH, name_extension='EDA')

+---------------------+------------+---------------+-------------+------------+-----------------+-----------------------+-------------+---------------------+--------------+-----------+----------+-----------------+---------------------+----------------------------------------+------------------+------------+------------------+-------------+-----------+-------------+-----------------+-------------+-------------------------+-------------+-------------+-------------+-------------+-------------+
| date                |   order_id | payment       | item_code   |   quantity |   gift_quantity |   unit_price_vat_excl |   unit_cogs |   unit_rrp_vat_excl | department   | channel   | owner    | site            | CreatedAt           | item_name                              | style            |   brand_id | brand_name       |   group0_id | group0    |   group1_id | group1          |   group2_id | group2                  | category    | gender      | age         | color       | size        |
|       

### A customized plot function to plot in Tableau style

In [10]:

reload(dc)


dc.plot_seasonality_boxplot(df, x='date', y='quantity', hue='group0', cycle='week_of_year', measure='sum')



<module 'src.modules.data_cleaning' from '/Users/ds/data_science/demand_prediction_data/src/modules/data_cleaning.py'>

## EAD on Weather Data

In [11]:
# feature enginnering:
df_FE = df.copy()

In [12]:
# please use the characters after the last dot in the site value as the country abbreviation
df_FE['country'] = df_FE['site'].str.split('.').str[-1]
df_FE['country'].unique()

array(['cz', 'de', 'sk', 'hu', 'ro', 'com', 'es', 'at', 'other', 'fr',
       'it', 'hr', 'nl', 'ie', 'be', 'dk', 'se', 'pt', 'pl', 'fi', 'si',
       'uk', 'bg'], dtype=object)

```python
Date Range:

Timestamp('2019-06-01 00:00:00')

Timestamp('2022-01-14 00:00:00')



country_mapping = {
    'cz': 'Czech Republic',
    'de': 'Germany',
    'sk': 'Slovakia',
    'hu': 'Hungary',
    'ro': 'Romania',
    'com': 'Commercial',  # Note: .com is not country-specific
    'es': 'Spain',
    'at': 'Austria',
    'other': 'Other',
    'fr': 'France',
    'it': 'Italy',
    'hr': 'Croatia',
    'nl': 'Netherlands',
    'ie': 'Ireland',
    'be': 'Belgium',
    'dk': 'Denmark',
    'se': 'Sweden',
    'pt': 'Portugal',
    'pl': 'Poland',
    'fi': 'Finland',
    'si': 'Slovenia',
    'uk': 'United Kingdom',
    'bg': 'Bulgaria'
}
capital_city_mapping = {
    'cz': 'Prague',
    'de': 'Berlin',
    'sk': 'Bratislava',
    'hu': 'Budapest',
    'ro': 'Bucharest',
    'com': 'N/A',  # Note: .com is not country-specific
    'es': 'Madrid',
    'at': 'Vienna',
    'other': 'N/A',
    'fr': 'Paris',
    'it': 'Rome',
    'hr': 'Zagreb',
    'nl': 'Amsterdam',
    'ie': 'Dublin',
    'be': 'Brussels',
    'dk': 'Copenhagen',
    'se': 'Stockholm',
    'pt': 'Lisbon',
    'pl': 'Warsaw',
    'fi': 'Helsinki',
    'si': 'Ljubljana',
    'uk': 'London',
    'bg': 'Sofia'
}

capital_coordinates_mapping = {
    'cz': (50.0755381, 14.4378005),
    'de': (52.5200066, 13.4049540),
    'sk': (48.1485965, 17.1077477),
    'hu': (47.497912, 19.040235),
    'ro': (44.4267674, 26.1025384),
    'com': 'N/A',  # Note: .com is not country-specific
    'es': (40.4167754, -3.7037902),
    'at': (48.2081743, 16.3738189),
    'other': 'N/A',
    'fr': (48.856614, 2.3522219),
    'it': (41.9027835, 12.4963655),
    'hr': (45.8150108, 15.9819190),
    'nl': (52.3702157, 4.8951679),
    'ie': (53.3498053, -6.2603097),
    'be': (50.8503396, 4.3517103),
    'dk': (55.6760968, 12.5683371),
    'se': (59.3293235, 18.0685808),
    'pt': (38.7222524, -9.1393366),
    'pl': (52.2296756, 21.0122287),
    'fi': (60.1698557, 24.9383791),
    'si': (46.0569465, 14.5057515),
    'uk': (51.5073509, -0.1277583),
    'bg': (42.6977082, 23.3218675)
}

```


In [13]:
dc.print_df_with_dtypes(df_FE, max_rows=200, print_on=True, data_generated_path=DATA_GENERATED_PATH, name_extension='EDA_FE')

+---------------------+------------+---------------+-------------+------------+-----------------+-----------------------+-------------+---------------------+--------------+-----------+----------+-----------------+---------------------+----------------------------------------+------------------+------------+------------------+-------------+-----------+-------------+-----------------+-------------+-------------------------+-------------+-------------+-------------+-------------+-------------+-----------+
| date                |   order_id | payment       | item_code   |   quantity |   gift_quantity |   unit_price_vat_excl |   unit_cogs |   unit_rrp_vat_excl | department   | channel   | owner    | site            | CreatedAt           | item_name                              | style            |   brand_id | brand_name       |   group0_id | group0    |   group1_id | group1          |   group2_id | group2                  | category    | gender      | age         | color       | size      

In [14]:
# Print the top 10 countries with the most sales
print('Top 10 countries with the most sales:')

# gouping by country, see which countries have the most sales
df_FE.groupby('country')['quantity'].sum().sort_values(ascending=False)



Top 10 countries with the most sales:


country
cz       726336
sk       367676
hu       293089
ro       285730
de       123385
es       120998
fr        74731
it        43002
hr        38192
com       37328
ie        17892
dk        15347
nl        11550
at        10988
se         8472
pt         5445
pl         4490
be         3622
fi         1787
si          712
other        70
uk           14
bg            6
Name: quantity, dtype: int64

In [15]:
# count the number of records from [cz, sk, hu, ro, de, es] is how many percent of the total number of records
df_FE[df_FE['country'].isin(['cz', 'sk'])].shape[0] / df_FE.shape[0] * 100
df_FE[df_FE['country'].isin(['cz', 'sk', 'hu'])].shape[0] / df_FE.shape[0] * 100
df_FE[df_FE['country'].isin(['cz', 'sk', 'hu', 'ro'])].shape[0] / df_FE.shape[0] * 100
df_FE[df_FE['country'].isin(['cz', 'sk', 'hu', 'ro', 'de'])].shape[0] / df_FE.shape[0] * 100
df_FE[df_FE['country'].isin(['cz', 'sk', 'hu', 'ro', 'de', 'es'])].shape[0] / df_FE.shape[0] * 100

49.997344831232304

63.41647379489409

76.40001409824124

82.15599703936934

87.75050812411152

In [16]:
dc.print_df_with_dtypes(df_FE, max_rows=200, print_on=True, data_generated_path=DATA_GENERATED_PATH, name_extension='EDA_FE')

+---------------------+------------+---------------+-------------+------------+-----------------+-----------------------+-------------+---------------------+--------------+-----------+----------+-----------------+---------------------+----------------------------------------+------------------+------------+------------------+-------------+-----------+-------------+-----------------+-------------+-------------------------+-------------+-------------+-------------+-------------+-------------+-----------+
| date                |   order_id | payment       | item_code   |   quantity |   gift_quantity |   unit_price_vat_excl |   unit_cogs |   unit_rrp_vat_excl | department   | channel   | owner    | site            | CreatedAt           | item_name                              | style            |   brand_id | brand_name       |   group0_id | group0    |   group1_id | group1          |   group2_id | group2                  | category    | gender      | age         | color       | size      

In [17]:
# # save to csv, continue EDA in Tableau
# df_FE.to_csv(DATA_GENERATED_PATH + 'df_FE.csv')

## Moon Phases





In [19]:
# read from the csv file data/Prague 2019-06-01 to 2022-01-14.csv

weather_Prague = pd.read_csv(DATA_ORIGINAL_PATH + 'Prague 2019-06-01 to 2022-01-14.csv')

# weather_Prague.datetime from object to datetime, the object format is 2019-06-01 
weather_Prague['datetime'] = pd.to_datetime(weather_Prague['datetime'])

# rename the column datetime to date, and set the index to date
weather_Prague.rename(columns={'datetime': 'date'}, inplace=True)
weather_Prague = weather_Prague.set_index('date')

# print weather_Prague
dc.print_df_with_dtypes(weather_Prague[['moonphase']].tail(30), max_rows=20, print_on=True, data_generated_path=DATA_GENERATED_PATH, name_extension='EDA_weather_Prague')


+---------------------+-------------+
| date                |   moonphase |
|                     |     float64 |
|                     |      NaN: 0 |
|---------------------+-------------|
| 2021-12-16 00:00:00 |        0.41 |
| 2021-12-17 00:00:00 |        0.44 |
| 2021-12-18 00:00:00 |        0.48 |
| 2021-12-19 00:00:00 |        0.5  |
| 2021-12-20 00:00:00 |        0.54 |
| 2021-12-21 00:00:00 |        0.58 |
| 2021-12-22 00:00:00 |        0.61 |
| 2021-12-23 00:00:00 |        0.65 |
| 2021-12-24 00:00:00 |        0.68 |
| 2021-12-25 00:00:00 |        0.71 |
| 2021-12-26 00:00:00 |        0.75 |
| 2021-12-27 00:00:00 |        0.75 |
| 2021-12-28 00:00:00 |        0.82 |
| 2021-12-29 00:00:00 |        0.85 |
| 2021-12-30 00:00:00 |        0.89 |
| 2021-12-31 00:00:00 |        0.92 |
| 2022-01-01 00:00:00 |        0.95 |
| 2022-01-02 00:00:00 |        0    |
| 2022-01-03 00:00:00 |        0.02 |
| 2022-01-04 00:00:00 |        0.06 |
+---------------------+-------------+


In [20]:


# read moonphase_future from data/prague 2022-01-01 to 2024-12-31.csv
moonphase_future = pd.read_csv(DATA_ORIGINAL_PATH + 'prague 2022-01-01 to 2024-12-31.csv')

# conver the moonphase_future.datetime from timestemp to datetime
# rename the column datetimeEpoch to datetime
moonphase_future = moonphase_future.rename(columns={'datetimeEpoch': 'datetime'})
moonphase_future['datetime'] = pd.to_datetime(moonphase_future['datetime'], unit='s')
moonphase_future['datetime'] = moonphase_future['datetime'] + pd.Timedelta(hours=1)

# rename the column datetime to date, and set the index to date
moonphase_future.rename(columns={'datetime': 'date'}, inplace=True)
moonphase_future = moonphase_future.set_index('date')

# print weather_Prague
dc.print_df_with_dtypes(moonphase_future[['moonphase']].tail(30), max_rows=20, print_on=True, data_generated_path=DATA_GENERATED_PATH, name_extension='EDA_moonphase_future')



+---------------------+-------------+
| date                |   moonphase |
|                     |     float64 |
|                     |      NaN: 0 |
|---------------------+-------------|
| 2024-12-02 00:00:00 |        0.04 |
| 2024-12-03 00:00:00 |        0.08 |
| 2024-12-04 00:00:00 |        0.11 |
| 2024-12-05 00:00:00 |        0.15 |
| 2024-12-06 00:00:00 |        0.18 |
| 2024-12-07 00:00:00 |        0.22 |
| 2024-12-08 00:00:00 |        0.25 |
| 2024-12-09 00:00:00 |        0.29 |
| 2024-12-10 00:00:00 |        0.33 |
| 2024-12-11 00:00:00 |        0.36 |
| 2024-12-12 00:00:00 |        0.4  |
| 2024-12-13 00:00:00 |        0.43 |
| 2024-12-14 00:00:00 |        0.47 |
| 2024-12-15 00:00:00 |        0.5  |
| 2024-12-16 00:00:00 |        0.53 |
| 2024-12-17 00:00:00 |        0.57 |
| 2024-12-18 00:00:00 |        0.6  |
| 2024-12-19 00:00:00 |        0.63 |
| 2024-12-20 00:00:00 |        0.66 |
| 2024-12-21 00:00:00 |        0.7  |
+---------------------+-------------+


The t-statistic of -3.342 indicates that the 'quantity' variable's mean for one moon phase group is lower than the other. The p-value of 0.00083 suggests that there is a significant difference between the means, allowing us to reject the null hypothesis.

### Hypotheses

1. **Group Differences:** Test if the `group_other` is different from each of the other groups (group_a, group_b, group_c, group_d, and group_e).

2. **Paired Group Differences:** Test if the difference between the following pairs of groups is not important:
    - group_a and group_b
    - group_b and group_c
    - group_c and group_d
    - group_e and group_a

3. **Mean Comparison:** Test if the mean of group_a, group_b, group_c, group_d, and group_e is much larger than the mean of `group_other`.

4. **Sum Comparison:** Test if the sum of the quantities from group_a, group_b, group_c, group_d, and group_e is much larger than the sum of the quantities from `group_other`.


In the given context, we aim to investigate the potential impact of moon phases on the daily sales quantity of a product. By carrying out these tests, we can gain insights into whether there is a statistically significant relationship between different moon phases and sales performance. This information can be valuable for understanding customer behavior and could potentially help with decision-making in areas such as marketing campaigns, pricing, and inventory management.

Specifically, we want to examine:

- If the sales quantity during specific moon phases (0, 0.25, 0.5, 0.75, and 1.0) is significantly different from the sales quantity during other moon phases.
- If there is a significant difference in sales quantity between specific moon phase pairs (e.g., full moon vs. new moon, first quarter vs. last quarter, etc.).
- If the mean sales quantity during the specific moon phases (0, 0.25, 0.5, 0.75, and 1.0) is much larger than the mean sales quantity during other moon phases.
- If the sum of sales quantity during the specific moon phases (0, 0.25, 0.5, 0.75, and 1.0) is much larger than the sum of sales quantity during other moon phases.

By conducting these hypothesis tests, we can determine if there is any evidence to support the idea that moon phases influence the daily sales quantity. If significant differences are found, this could provide a basis for further investigation and potentially inform business strategies that take into account the effects of moon phases on sales.


| Group   | Moon Phase   | Description                                                                                               |
|---------|--------------|-----------------------------------------------------------------------------------------------------------|
| Group_0 | Other values | Contains all observations with moon phase values not equal to 0, 0.25, 0.5, 0.75, or 1.                    |
| Group_1 | 0            | Contains all observations with moon phase values close to 0 (New Moon).                                   |
| Group_2 | 0.25         | Contains all observations with moon phase values close to 0.25 (First Quarter / Waxing Half Moon).        |
| Group_3 | 0.5          | Contains all observations with moon phase values close to 0.5 (Full Moon).                                |
| Group_4 | 0.75         | Contains all observations with moon phase values close to 0.75 (Last Quarter / Waning Half Moon).         |
| Group_5 | 1            | Contains all observations with moon phase values close to 1 (New Moon, same as Group_1 but a different set). |


In [21]:
# import ttest_ind
from scipy.stats import ttest_ind
import numpy as np

from tabulate import tabulate

def moonphase_ttest_v2(df_FE, weather_Prague, moonphase_values=[0, 0.25, 0.5, 0.75, 1.0], tolerance=2.5e-2):
    # Set the datetime column as the index in weather_Prague
    # weather_Prague = weather_Prague.set_index('date')
    
    # Merge moonphase data to df_FE
    df_FE = df_FE.merge(weather_Prague[['moonphase']], left_index=True, right_index=True, how='left')

    # Create a mask for specific moon phase values
    specific_moonphase_mask = np.zeros(df_FE.shape[0], dtype=bool)
    for value in moonphase_values:
        specific_moonphase_mask |= np.isclose(df_FE['moonphase'], value, atol=tolerance)

    # Create groups based on moon phase values
    groups = {i: df_FE[np.isclose(df_FE['moonphase'], value, atol=tolerance)] for i, value in enumerate(moonphase_values, 1)}
    groups[0] = df_FE[~specific_moonphase_mask]
    
    # Compare the means of all pairs of groups and display the results in a table
    comparisons = []
    for i, group_i in groups.items():
        for j, group_j in groups.items():
            if i < j:
                t_statistic, p_value = ttest_ind(
                    group_i['quantity'],
                    group_j['quantity'],
                    equal_var=False,
                    nan_policy='omit'
                )
                comparisons.append([f"Group_{i}", f"Group_{j}", t_statistic, p_value])
    
    print(tabulate(comparisons, headers=["Group A", "Group B", "t-statistic", "p-value"], tablefmt="grid"))

    # write the result to a txt file, in tabulate format
    with open(DATA_GENERATED_PATH + 'moonphase_test_v2.txt', 'w') as f:
        f.write(tabulate(comparisons, headers=["Group A", "Group B", "t-statistic", "p-value"], tablefmt="grid"))
        

# Example usage
moonphase_ttest_v2(df_FE, weather_Prague)


+-----------+-----------+---------------+-------------+
| Group A   | Group B   |   t-statistic |     p-value |
+===========+===========+===============+=============+
| Group_1   | Group_2   |      2.76067  | 0.00576904  |
+-----------+-----------+---------------+-------------+
| Group_1   | Group_3   |     10.7539   | 5.79286e-27 |
+-----------+-----------+---------------+-------------+
| Group_1   | Group_4   |      2.17868  | 0.0293571   |
+-----------+-----------+---------------+-------------+
| Group_1   | Group_5   |     -2.79066  | 0.00526435  |
+-----------+-----------+---------------+-------------+
| Group_2   | Group_3   |     10.4534   | 1.43283e-25 |
+-----------+-----------+---------------+-------------+
| Group_2   | Group_4   |     -0.714977 | 0.474624    |
+-----------+-----------+---------------+-------------+
| Group_2   | Group_5   |     -4.44735  | 8.74534e-06 |
+-----------+-----------+---------------+-------------+
| Group_3   | Group_4   |    -10.8633   | 1.7524

## Using Week of the Month as Proxy Variables

When working with weather data, one approach to simplifying the analysis is to use the week of the month as proxy variables. This means aggregating the data by week, allowing us to analyze the differences in weather patterns and identify any significant changes over time. However, it's important to carefully consider the implications of this approach, as it may not fully capture the nuances of the actual weather conditions.

### Hypothesis Testing

To test the validity of using the week of the month as proxy variables, we can perform a Kruskal-Wallis test for numerical features and a Chi-square test for categorical features. These tests help us determine whether the means (or distributions, in the case of categorical variables) are significantly different across different weeks of the month.

We can set up the null hypothesis (H0) as follows:

- H0: There is no significant difference in means (or distributions) of the weather features across different weeks of the month.

If the p-value obtained from the test is greater than the significance level (alpha), typically set to 0.05, we fail to reject the null hypothesis, suggesting that there is no significant difference in the means (or distributions) of the weather features across different weeks of the month.

### Results and Interpretation

After performing the hypothesis tests, we can calculate the proportion of weeks with no significant difference in means (or distributions) for each weather feature. Features with a proportion greater than the significance level (e.g., 0.05) can be considered as suitable proxy variables for weather data. However, it's important to note that this is a simplification and may not perfectly represent the actual weather conditions.

Based on the test results, we can decide which features to keep in our analysis and which ones to drop due to their low proportion values (less than or equal to 0.05). This process can help us create a simplified dataset that maintains relevant information while reducing complexity.



In [22]:
dc.print_df_with_dtypes(weather_Prague.head(), max_rows=20, print_on=True, data_generated_path=DATA_GENERATED_PATH, name_extension='EDA_weather_Prague')

+---------------------+----------+-----------+--------------------------------------------+------------+-------------+-----------+-----------+-----------+----------------+----------------+-------------+-----------+------------+-----------+--------------+---------------+--------------+-----------+-------------+------------+-------------+-----------+--------------------+--------------+--------------+------------------+---------------+-----------+--------------+---------------------+---------------------+-------------+------------------+--------------------------------------+-------------------+----------+
| date                | name     | address   | resolvedAddress                            |   latitude |   longitude |   tempmax |   tempmin |      temp |   feelslikemax |   feelslikemin |   feelslike |       dew |   humidity |    precip |   precipprob |   precipcover |   preciptype |      snow |   snowdepth |   windgust |   windspeed |   winddir |   sealevelpressure |   cloudcover |   

In [23]:
weather_col = ['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'preciptype', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex', 'severerisk', 'sunrise', 'sunset',
       'moonphase', 'conditions']

In [24]:
import pandas as pd
from scipy.stats import kruskal, chi2_contingency

# List the categorical features
categorical_features = ['preciptype', 'conditions']

# Define the function to perform the Kruskal-Wallis and Chi-square tests
def kruskal_wallis_chi_square_test(weather_data, weeks_range=(1, 53), features=weather_col):
    weather_data.index = pd.to_datetime(weather_data.index)
    weather_data['week_of_year'] = weather_data.index.isocalendar().week
    weather_data['year'] = weather_data.index.year

    results = []

    for week in range(weeks_range[0], weeks_range[1] + 1):
        for feature in features:
            data_by_week = weather_data[weather_data['week_of_year'] == week]
            groups = data_by_week.groupby('year')[feature].apply(list)
            
            if len(groups) < 2:  # At least two groups are needed for the test
                continue

            if feature not in categorical_features:
                # Skip the test if all values in the groups are identical
                if all(len(set(group)) == 1 and group[0] == groups.iloc[0][0] for group in groups):
                    continue
                
                # Perform Kruskal-Wallis test for numerical features
                test_statistic, p_value = kruskal(*groups)
            else:
                # Create a contingency table for categorical features
                contingency_table = pd.crosstab(data_by_week['year'], data_by_week[feature])

                # Perform Chi-square test for categorical features
                test_statistic, p_value, _, _ = chi2_contingency(contingency_table)

            results.append((week, feature, test_statistic, p_value))

    results_df = pd.DataFrame(results, columns=['Week', 'Feature', 'Test_statistic', 'p_value'])
    return results_df

# Run the Kruskal-Wallis and Chi-square tests
results_df = kruskal_wallis_chi_square_test(weather_Prague)

# Set a significance level (alpha)
alpha = 0.05

# Calculate the proportion of weeks with no significant difference for each feature
results_df['no_significant_difference'] = results_df['p_value'] > alpha
proportion_no_difference = results_df.groupby('Feature')['no_significant_difference'].mean()

print("Proportion of weeks with no significant difference in means for each feature:")
print(proportion_no_difference)


Proportion of weeks with no significant difference in means for each feature:
Feature
cloudcover          0.679245
conditions          0.754717
dew                 0.641509
feelslike           0.490566
feelslikemax        0.528302
feelslikemin        0.584906
humidity            0.641509
moonphase           0.301887
precip              0.773585
precipcover         0.773585
precipprob          0.735849
preciptype          0.924528
sealevelpressure    0.471698
severerisk          0.000000
snow                0.818182
snowdepth           0.700000
solarenergy         0.849057
solarradiation      0.830189
sunrise             0.000000
sunset              0.000000
temp                0.452830
tempmax             0.528302
tempmin             0.603774
uvindex             0.867925
visibility          0.622642
winddir             0.811321
windgust            0.000000
windspeed           0.641509
Name: no_significant_difference, dtype: float64


In [25]:
def save_results_to_file(proportion_no_difference, filename=DATA_GENERATED_PATH + 'kruskal_wallis_test_all.txt'):
    with open(filename, 'w') as file:
        file.write("Proportion of weeks with no significant difference in means for each feature:\n")
        for feature, proportion in proportion_no_difference.items():
            file.write(f"{feature:<20}{proportion:.6f}\n")


# Save the results to a file
save_results_to_file(proportion_no_difference)

drop not useful columns from the weather_Prague, based on the hypothesis test results

In [26]:
# Find the columns with proportion less than or equal to 0.05
columns_to_drop = proportion_no_difference[proportion_no_difference <= 0.05].index.tolist()

# Drop those columns from the weather_Prague DataFrame
weather_Prague_simplified = weather_Prague.drop(columns_to_drop, axis=1)

# Print the remaining columns
print("Remaining columns in weather_Prague_simplified:")
print(weather_Prague_simplified.columns)


Remaining columns in weather_Prague_simplified:
Index(['name', 'address', 'resolvedAddress', 'latitude', 'longitude',
       'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'preciptype', 'snow', 'snowdepth', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex', 'moonphase', 'conditions', 'description',
       'icon', 'source', 'week_of_year', 'year'],
      dtype='object')


In [27]:
# Feature Engineering

In [28]:
dc.print_df_with_dtypes(df_FE, max_rows=20, print_on=True, data_generated_path=DATA_GENERATED_PATH, name_extension='EDA_df_FE')

+---------------------+------------+------------+-------------+------------+-----------------+-----------------------+-------------+---------------------+--------------+-----------+----------+-----------------+---------------------+-----------------------+-------------+------------+-----------------+-------------+-----------+-------------+----------------+-------------+--------------------+-------------+-------------+-------------+-------------+-------------+-----------+
| date                |   order_id | payment    | item_code   |   quantity |   gift_quantity |   unit_price_vat_excl |   unit_cogs |   unit_rrp_vat_excl | department   | channel   | owner    | site            | CreatedAt           | item_name             | style       |   brand_id | brand_name      |   group0_id | group0    |   group1_id | group1         |   group2_id | group2             | category    | gender      | age         | color       | size        | country   |
|                     |     string | object     

In [36]:
# Filter the DataFrame based on the "country" column
filtered_df = df_FE[df_FE["country"] == "cz"]

# Group by "group1" column and sum the "quantity" column, then sort values in descending order
filtered_df.groupby("group1")["quantity"].sum().sort_values(ascending=False)

# Group by "group2" column and sum the "quantity" column, then sort values in descending order
filtered_df.groupby("group2")["quantity"].sum().sort_values(ascending=False)


group1
unspecified        289613
Pants               65732
Socks               61210
T-Shirts            54987
Running shoes       52124
Football shoes      44135
Sweatshirts         25138
Other Footwear      20648
Jerseys             14034
Jackets             12834
Other Apparel       10220
Other Equipment      9988
Balls                7529
Gloves               7182
Bags                 5795
Guards               5316
Bras                 5068
Backpacks            4935
Underwear            4561
Slides               3802
Fitness Shoes        3483
Caps                 3100
Training             2917
Tracksuits           2862
Beanies              2328
Other nutrition      1750
Gymsacks             1232
Bar                   632
Other                 592
Vests                 483
Hydrate               389
Glasses               381
Lamps                 365
Skirts                251
Pumps                 195
Sports gloves         136
Dress                 134
Earphones             132
Sport

group2
unspecified                592272
Shorts                      36035
With hood                   18724
Longsleeved (LS) shirts     15746
Football socks              15681
Trail                       13124
Spikes                       7170
Compression shirts           6424
Tank                         5011
Indoor/Court (IC)            4701
Compression pants            3369
Football gloves              3072
43924                        2026
Polo                         1957
Sleeveless (SL) shirts        888
Fitness gloves                136
Name: quantity, dtype: int64

In [ ]:
# aggreate by the values from gorup2 : [
# Shorts                      
# With hood                   
# Trail                       
# Longsleeved (LS) shirts     
# Spikes                      ]
# daily sum

# group1: 'Pants', 'Socks';
# group2: 'Shorts' 'With hood';

# Feature Engineering:

In [29]:
dc.print_df_with_dtypes(df_FE, max_rows=20, print_on=True, data_generated_path=DATA_GENERATED_PATH, name_extension='EDA_df_FE')

+---------------------+------------+------------+-------------+------------+-----------------+-----------------------+-------------+---------------------+--------------+-----------+----------+-----------------+---------------------+-----------------------+-------------+------------+-----------------+-------------+-----------+-------------+----------------+-------------+--------------------+-------------+-------------+-------------+-------------+-------------+-----------+
| date                |   order_id | payment    | item_code   |   quantity |   gift_quantity |   unit_price_vat_excl |   unit_cogs |   unit_rrp_vat_excl | department   | channel   | owner    | site            | CreatedAt           | item_name             | style       |   brand_id | brand_name      |   group0_id | group0    |   group1_id | group1         |   group2_id | group2             | category    | gender      | age         | color       | size        | country   |
|                     |     string | object     

### Pivot table for all unique values in group1 and group2

In [38]:
import pandas as pd

# Assuming df_FE is already loaded into memory
# Create separate dataframes for group1 and group2
df_group1 = df_FE[df_FE['group1_id'].notnull()]
df_group2 = df_FE[df_FE['group2_id'].notnull()]

# Pivot tables for each group
pivot_group1 = df_group1.pivot_table(index='date', columns='group1', values='quantity', aggfunc='sum', fill_value=0)
pivot_group2 = df_group2.pivot_table(index='date', columns='group2', values='quantity', aggfunc='sum', fill_value=0)

# Merge the pivot tables on the index (date)
result = pd.merge(pivot_group1, pivot_group2, left_index=True, right_index=True, how='outer', suffixes=('_group1', '_group2'))

# Fill NaN values with 0
result.fillna(0, inplace=True)

# Rename columns as required
result.columns = [f'group1_{col}_group1' if '_group1' in col else f'group2_{col}_group2' for col in result.columns]

dc.print_df_with_dtypes(result, max_rows=20, print_on=True, data_generated_path=DATA_GENERATED_PATH, name_extension='EDA_pivot_result')

+---------------------+---------------------------+----------------------+-----------------------+---------------------+-------------------------+----------------------+----------------------+-----------------------+---------------------------+-------------------------------+--------------------------------+-------------------------+------------------------+------------------------+--------------------------+-------------------------+-------------------------+-------------------------+-----------------------+-----------------------+-------------------------------+---------------------------------+--------------------------------+---------------------------------+-----------------------+-----------------------+-------------------------------+------------------------+------------------------+-----------------------+-------------------------------+------------------------------+-----------------------------+--------------------------+-----------------------------+-------------------------

### Pivot table for the four values only:

group1: 'Pants', 'Socks';
group2: 'Shorts' 'With hood';

In [43]:
import pandas as pd

# Assuming df_FE is already loaded into memory
# Create separate dataframes for group1 and group2
df_group1 = df_FE[df_FE['group1_id'].notnull()]
df_group2 = df_FE[df_FE['group2_id'].notnull()]

# Pivot tables for each group
pivot_group1 = df_group1.pivot_table(index='date', columns='group1', values='quantity', aggfunc='sum', fill_value=0)
pivot_group2 = df_group2.pivot_table(index='date', columns='group2', values='quantity', aggfunc='sum', fill_value=0)


# Select the required columns
pivot_group1 = pivot_group1[['Pants', 'Socks']]
pivot_group2 = pivot_group2[['Shorts', 'With hood']]

# Merge the pivot tables on the index (date)
result = pd.merge(pivot_group1, pivot_group2, left_index=True, right_index=True, how='outer')

# Fill NaN values with 0
result.fillna(0, inplace=True)

# Rename columns as required
result.columns = [f'group1_{col}' if col in ['Pants', 'Socks'] else f'group2_{col}' for col in result.columns]

print(result)


### Do this only for the country CZ



In [56]:
import pandas as pd

# Assuming df_FE is already loaded into memory
# Filter the data for the country 'cz'
df_FE_cz = df_FE[df_FE['country'] == 'cz']

# Create separate dataframes for group1 and group2
df_group1_cz = df_FE_cz[df_FE_cz['group1_id'].notnull()]
df_group2_cz = df_FE_cz[df_FE_cz['group2_id'].notnull()]

# Pivot tables for each group
pivot_group1_cz = df_group1_cz.pivot_table(index='date', columns='group1', values='quantity', aggfunc='sum', fill_value=0)
pivot_group2_cz = df_group2_cz.pivot_table(index='date', columns='group2', values='quantity', aggfunc='sum', fill_value=0)

# Select the required columns
pivot_group1_cz = pivot_group1_cz[['Pants', 'Socks']]
pivot_group2_cz = pivot_group2_cz[['Shorts', 'With hood']]

# Merge the pivot tables on the index (date)
result_cz = pd.merge(pivot_group1_cz, pivot_group2_cz, left_index=True, right_index=True, how='outer')

# Fill NaN values with 0
result_cz.fillna(0, inplace=True)

# Rename columns as required
result_cz.columns = [f'group1_{col}' if col in ['Pants', 'Socks'] else f'group2_{col}' for col in result_cz.columns]

print(result_cz)


            group1_Pants  group1_Socks  group2_Shorts  group2_With hood
date                                                                   
2019-06-01             8             3              5                 1
2019-06-02             1             0              1                 0
2019-06-03             5             1              4                 0
2019-06-04           109            34             75                 6
2019-06-05           154            87            109                41
...                  ...           ...            ...               ...
2022-01-10            89            99             27                22
2022-01-11            74           119             30                17
2022-01-12            85            83             35                20
2022-01-13            69            79             40                12
2022-01-14            20            38              9                 1

[957 rows x 4 columns]


In [ ]:
# create  2019-06-08 00:00:00 record and ffill



In [47]:
dc.print_df_with_dtypes(result_cz, max_rows=20, print_on=True, data_generated_path=DATA_GENERATED_PATH, name_extension='EDA_result_cz')

+---------------------+----------------+----------------+-----------------+--------------------+
| date                |   group1_Pants |   group1_Socks |   group2_Shorts |   group2_With hood |
|                     |          int64 |          int64 |           int64 |              int64 |
|                     |         NaN: 0 |         NaN: 0 |          NaN: 0 |             NaN: 0 |
|---------------------+----------------+----------------+-----------------+--------------------|
| 2019-06-01 00:00:00 |              8 |              3 |               5 |                  1 |
| 2019-06-02 00:00:00 |              1 |              0 |               1 |                  0 |
| 2019-06-03 00:00:00 |              5 |              1 |               4 |                  0 |
| 2019-06-04 00:00:00 |            109 |             34 |              75 |                  6 |
| 2019-06-05 00:00:00 |            154 |             87 |             109 |                 41 |
| 2019-06-06 00:00:00 |       

In [57]:
# Forward-fill missing record for the specific date 2019-06-09
result_cz = result_cz.asfreq('D', fill_value=0)
result_cz.loc['2019-06-09'] = result_cz.loc['2019-06-08']

In [59]:
dc.print_df_with_dtypes(result_cz, max_rows=20, print_on=True, data_generated_path=DATA_GENERATED_PATH, name_extension='EDA_result_cz_ffilled')

+---------------------+----------------+----------------+-----------------+--------------------+
| date                |   group1_Pants |   group1_Socks |   group2_Shorts |   group2_With hood |
|                     |          int64 |          int64 |           int64 |              int64 |
|                     |         NaN: 0 |         NaN: 0 |          NaN: 0 |             NaN: 0 |
|---------------------+----------------+----------------+-----------------+--------------------|
| 2019-06-01 00:00:00 |              8 |              3 |               5 |                  1 |
| 2019-06-02 00:00:00 |              1 |              0 |               1 |                  0 |
| 2019-06-03 00:00:00 |              5 |              1 |               4 |                  0 |
| 2019-06-04 00:00:00 |            109 |             34 |              75 |                  6 |
| 2019-06-05 00:00:00 |            154 |             87 |             109 |                 41 |
| 2019-06-06 00:00:00 |       

In [60]:
# save the result_cz to csv
result_cz.to_csv(DATA_GENERATED_PATH + 'EDA_result_cz_ffilled.csv')

In [61]:
result_cz.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 959 entries, 2019-06-01 to 2022-01-14
Freq: D
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   group1_Pants      959 non-null    int64
 1   group1_Socks      959 non-null    int64
 2   group2_Shorts     959 non-null    int64
 3   group2_With hood  959 non-null    int64
dtypes: int64(4)
memory usage: 69.8 KB
